In [43]:
#pip install --upgrade -q openai langchain langchain-openai langchain-community langgraph

In [15]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

In [16]:
OPENAI_MODEL = "gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [17]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, tool
from typing import List
import json
import os
import select
from datetime import datetime
from textwrap import dedent
from typing import Dict, List, Optional, TypeVar, Union
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic import ValidationError
from typing import Annotated, Any, Dict, List, Literal, Optional, TypedDict, TypeVar, Union
from IPython.display import Image, display
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.runnables import Runnable, RunnableConfig, RunnableLambda
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
import sys
import uuid
from pprint import pprint

In [18]:
prompt_template = dedent("""
### INSTRUÇÕES PARA RESPONDER À PERGUNTA DO HUMANO:

1- Entenda a DINÂMICA:

    Você é um vendedor online que responde às perguntas do cliente usando o histórico da conversa e as ferramentas disponíveis. 

2. Passos PARA RESPONDER:

    Pergunta: Pergunta do cliente.
    Reflexão: 
        a. Pense passo a passo
        b. Analise a pergunta do cliente.
        c. Revise o histórico para contexto.
        d. Verifique as ferramentas disponíveis.
    Ação: A ação a ser tomada. 
    Entrada: Dados necessários para a ação 
    Observação: Resultado da ação 
    Reflexão: Repita se necessário
    Resposta Final: Resposta para a pergunta do cliente

### Contexto:
- Idioma da Resposta: {idioma}
- E-mail do Usuário: {email_usuario}
- Data de Hoje: {data_hoje}
- Dia da Semana: {dia_da_semana}

### Histórico da conversa:
{chat_history} 

### Solicitação do cliente:
{input}

Scratchpad:
{agent_scratchpad}

===
{messages}
===

**Begin!**

Resposta:
""")


In [19]:
prompt_str = dedent("""Você é um assistente de compras atencioso, educado e prestativo.
Sua missão é responder às perguntas do cliente com gentileza, cortesia e precisão.

## Regras de Resposta

1. Sempre responda com a lista completa de produtos solicitados, incluindo descrições e preços detalhados. Não omita itens nem forneça exemplos parciais.
2. Todas as informações devem ser baseadas em dados concretos. Nunca invente produtos ou preços.
3. Se um produto não for encontrado, informe quais itens estão ausentes, mas retorne todos os outros produtos disponíveis.
4. Caso nenhum produto seja encontrado, informe claramente e sugira ao cliente verificar abreviações ou erros nos nomes.
5. Baseie suas respostas exclusivamente no HISTÓRICO DA CONVERSA e CONTEXTO fornecido ou nas ferramentas disponíveis.
6. Não procure ou especule informações fora do histórico ou contexto. Se não houver dados suficientes, responda:
"Desculpe, não tenho informações suficientes para responder a essa pergunta com precisão."
7. Mantenha um tom cordial, educado e informativo. Forneça respostas claras e diretas.

## Instruções para Responder

Analise o histórico da conversa e responda a solicitação do cliente.

# Contexto:
- Idioma da Resposta: português
- E-mail do Usuário: mrctito@gmail.com
- Data de Hoje: 2024-12-17
- Dia da Semana: terça

# Histórico da conversa:
Cliente: Pergunta do Humano:
como está o clima?
Assistente: None
 
Cliente: Pergunta do Humano:
como está o clima?
Assistente: None
 

# Solicitação do cliente:
como está o clima?

---

# Scratchpad:


---

# Mensagens:  
{{messages}}

---

Resposta:""")

In [46]:
query = "como está o clima?"
idioma = "português"
codigo_cliente = 1
email_cliente = "mrctito@gmail.com"
nome_cliente = "Paulo"
email_usuario = "mrctito@gmail.com"
nome_usuario = "Paulo"
memory_messages = None

contexto = {
    "messages": [HumanMessage(content=query)],
    "input": query,
    "idioma": idioma,
    "dia_da_semana": datetime.now().strftime("%A"),
    "data_hoje": datetime.now().strftime("%Y-%m-%d"),
    "email_cliente": email_usuario,
    "nome_cliente": nome_usuario,
    "email_usuario": email_usuario,
    "nome_usuario": nome_usuario,
    "chat_history": memory_messages if memory_messages else " ",
    "agent_scratchpad": "",
}

In [47]:
@tool
async def tool_informa_o_clima() -> str:
    """
    Útil para informar o clima atual.
    """
    return ["O clima está estável com sol."]

In [48]:
tools = []
tools.append(tool_informa_o_clima)
tools_names = "\n\n".join([f"Ferramenta: {tool.name}\nDescrição: {tool.description}" for tool in tools])
tool_node = ToolNode(tools)

In [49]:
llm = ChatOpenAI(model=OPENAI_MODEL,
                 api_key=OPENAI_API_KEY,
                 temperature=0.7,
                 verbose=True)

In [50]:
llm_with_tools = llm.bind_tools(tools)

In [51]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_str),
        ("human", "{input}"),
        ("placeholder", "{messages}"),
    ]
)

In [52]:
assistant_runnable = primary_assistant_prompt | llm_with_tools

In [53]:
# Define State
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [54]:
# Define the function that determines whether to continue or not
def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    else:
        return "tools"


# Define the function that calls the model
async def call_model(state: State, config: RunnableConfig):
    messages = state["messages"]
    # Note: Passing the config through explicitly is required for python < 3.11
    # Since context var support wasn't added before then: https://docs.python.org/3/library/asyncio-task.html#creating-tasks

    configuration = config.get("configurable", {})
    contexto = configuration.get("contexto", {})
    user_id = configuration.get("user_id", None)
    state = {**state, "user_info": user_id, **contexto}
    
    response = await assistant_runnable.ainvoke(messages, config)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next we pass in the path map - all the nodes this edge could go to
    ["tools", END],
)

workflow.add_edge("tools", "agent")

In [55]:
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
thread_id = str(uuid.uuid4())
config = {
    "configurable": {
        # fetch the user's id
        "user_id": email_usuario,
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
        "contexto": contexto
    }
}

In [56]:
resultado = await graph.ainvoke(contexto, config, debug=True)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'agent_scratchpad': '',
 'chat_history': ' ',
 'data_hoje': '2024-12-18',
 'dia_da_semana': 'Wednesday',
 'email_cliente': 'mrctito@gmail.com',
 'email_usuario': 'mrctito@gmail.com',
 'idioma': 'português',
 'input': 'como está o clima?',
 'messages': [HumanMessage(content='como está o clima?', additional_kwargs={}, response_metadata={})],
 'nome_cliente': 'Paulo',
 'nome_usuario': 'Paulo'}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='como está o clima?', additional_kwargs={}, response_metadata={})]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='como está o clima?', additional_kwargs={}, response_metadata={}, id='1b1691ab-ccff-4e85-9a8e-d71f067162ad')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'messages': [HumanMessage(content='como está o clima?', additional_kwargs={}, response_metadata

In [45]:
pprint(resultado)

{'messages': [HumanMessage(content='como está o clima?', additional_kwargs={}, response_metadata={}, id='50d48c09-fe4e-4f68-b3d3-b048b68c805d'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZXqfzfc7fKkfessWqXYsm4Tw', 'function': {'arguments': '{}', 'name': 'tool_informa_o_clima'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 460, 'total_tokens': 474, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-931165a6-3c46-4fd7-b470-5a1d9474a8a4-0', tool_calls=[{'name': 'tool_informa_o_clima', 'args': {}, 'id': 'call_ZXqfzfc7fKkfessWqXYsm4Tw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 46